In [1]:
from langchain.document_loaders.csv_loader import CSVLoader 
from langchain_community.document_loaders.directory import DirectoryLoader

In [2]:
def load_pdf(data):
    loader = DirectoryLoader(data, 
                             glob='*.csv', 
                             loader_cls=CSVLoader)
    documents = loader.load()
    return documents

In [3]:
extracted_data = load_pdf(r"E:\projects\EduBotIQ\Data Prepration")

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 

#Create text chunk
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 2847


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [7]:
embeddings = download_hugging_face_embeddings()

C:\Users\harik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [9]:
from langchain_community.vectorstores import FAISS
def create_db(text_chunks, embeddings):
    DB_FAISS_PATH = 'vectorstore/db_faiss'
    
    db = FAISS.from_documents(text_chunks, embeddings)
    db.save_local(DB_FAISS_PATH)

In [10]:
create_db(text_chunks, embeddings)